In [1]:
//import org.apache.spark.SparkContext
//val sc = new SparkContext("local", "")

//NOTE:
//Running this with default settings, I got an OutOfMemory Error.
// A bit of digging revealed a fix was to set the env var `SPARK_OPTS=--driver-memory=2g` (up from the 1g default)
// before running `jupyter lab` (then the kernel recieves this value and inserts it into the run command).
// See: https://stackoverflow.com/a/26327528 for similar q 
// Var names / how applied at https://toree.apache.org/docs/current/user/installation/ under "Configuring Spark > SPARK_OPTS"
// (note using --executor-memory=Xg may not help for local mode execution: https://stackoverflow.com/a/26578005)


Name: Syntax Error.
Message: 
StackTrace: 

In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
  .builder()
  .appName("abc_news_text_classification")
  //.config("spark.some.config.option", "some-value")
  .getOrCreate()
val sc = spark.sparkContext

sc = org.apache.spark.SparkContext@74c9c906


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@710c0c62


org.apache.spark.SparkContext@74c9c906

In [3]:

// Read the csv file

// Specify the schema
import org.apache.spark.sql.types._

val newsCSVSchema = StructType(Array(
    StructField("category", StringType, true),
    StructField("title", StringType, true),
    StructField("description", StringType, true),
    StructField("guid", StringType, true),
    StructField("published", StringType, true))
)

val newsData = spark.read.format("csv")
    .option("sep", ",")
    .option("quote", "\"").option("escape", "\"") //This is for the quote char in a string line, see https://stackoverflow.com/a/45138591
    //.option("inferSchema", "true")
    .schema(newsCSVSchema)
    .option("header", "false") //true")
    .load("news_data.csv")


newsData = [category: string, title: string ... 3 more fields]


newsCSVSchema: org.apache.spark.sql.types.StructType = StructType(StructField(category,StringType,true), StructField(title,StringType,true), StructField(description,StringType,true), StructField(guid,StringType,true), StructField(published,StringType,true))


[category: string, title: string ... 3 more fields]

In [4]:
newsData.cache.persist
println(s"Data has ${newsData.count} observations")
newsData.show()

Data has 2000 observations
+--------+--------------------+--------------------+--------------------+--------------------+
|category|               title|         description|                guid|           published|
+--------+--------------------+--------------------+--------------------+--------------------+
|Business|'Easy debt to get...|Butcher Samantha ...|https://www.abc.n...|Sat, 18 Jan 2020 ...|
|Business|Canada the latest...|From Canada to As...|https://www.abc.n...|Sat, 18 Jan 2020 ...|
|Business|No sheep bottoms ...|Not all wool prod...|https://www.abc.n...|Sat, 18 Jan 2020 ...|
|Business|A 24-year-old cou...|A landmark trial ...|https://www.abc.n...|Sat, 18 Jan 2020 ...|
|Business|Chinese birth rat...|The country's bir...|https://www.abc.n...|Fri, 17 Jan 2020 ...|
|Business|'Can I have some ...|Australian teleco...|https://www.abc.n...|Fri, 17 Jan 2020 ...|
|Business|Restored 1940s ai...|From an aircraft ...|https://www.abc.n...|Fri, 17 Jan 2020 ...|
|Business|'Canberra, co

In [5]:
val newsTrainDevTest = newsData.randomSplit(Array(0.7, 0.2, 0.1), seed = 100)
//seed for repeatability

newsTrainDevTest = Array([category: string, title: string ... 3 more fields], [category: string, title: string ... 3 more fields], [category: string, title: string ... 3 more fields])


Array([category: string, title: string ... 3 more fields], [category: string, title: string ... 3 more fields], [category: string, title: string ... 3 more fields])

In [6]:
val newsTrainDF = newsTrainDevTest(0);
val newsDevDF = newsTrainDevTest(1);

newsDevDF = [category: string, title: string ... 3 more fields]


newsTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, title: string ... 3 more fields]


[category: string, title: string ... 3 more fields]

In [7]:
// My plan is to tokenise, remove stop words, count vectorise (sparse), and use a simple classification
// (optionally consider tf-idf)

Name: Syntax Error.
Message: 
StackTrace: 

In [8]:
import org.apache.spark.ml.feature.{RegexTokenizer, Tokenizer}

val regexTokenizer = new RegexTokenizer()
    .setInputCol("title")
    .setOutputCol("title_words")
    .setPattern("\\W")


regexTokenizer = regexTok_b121ea98bfd6


regexTok_b121ea98bfd6

In [9]:
import org.apache.spark.ml.feature.StopWordsRemover

val stopWordRemover = new StopWordsRemover()
  .setInputCol(regexTokenizer.getOutputCol)//eg "raw")
  .setOutputCol("titleFiltered")
//optionally add stopwords "english"

stopWordRemover = stopWords_9f2bafbfb9b0


stopWords_9f2bafbfb9b0

In [10]:
import org.apache.spark.ml.feature.HashingTF

val hashingTF = new HashingTF()
    .setInputCol(stopWordRemover.getOutputCol)
    .setOutputCol("hashVectorTF")
    //.setNumFeatures(scala.math.pow(2,xx).asInstanceOf[Int])

hashingTF = hashingTF_ea018de37b84


hashingTF_ea018de37b84

In [11]:
hashingTF.getNumFeatures

262144

In [12]:
// Since hashingTF is the only input we want here, it is ok to stop
// the transform parts of the pipeline here.
// We could use eg VectorAssembler otherwise

Name: Syntax Error.
Message: 
StackTrace: 

In [13]:
// Note though that to use LogisticRegression, we must convert the
// string labels to numeric.
import org.apache.spark.ml.feature.StringIndexer

val categoryIndexer = new StringIndexer()
  .setInputCol("category")
  .setOutputCol("categoryIndex")


categoryIndexer = strIdx_fadf8e53504a


strIdx_fadf8e53504a

In [14]:
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression()

//set any optional params:
//lr.setMaxIter(i)
//lr.setRegParam(r)
//...

lr.setFeaturesCol(hashingTF.getOutputCol)
lr.setLabelCol(categoryIndexer.getOutputCol)
//optionally: lr.setFamily("multinomial")

//lr.setPredictionCol("")


lr: org.apache.spark.ml.classification.LogisticRegression = logreg_d55e66b6da97


logreg_d55e66b6da97

In [15]:
// And after a prediction, we want to get back to the real string category
// Though I am not sure how to do this prior, so will do it post fit (extracting labels out of the pipeline)
/*
import org.apache.spark.ml.feature.IndexToString

val predictionIdxNamer = new IndexToString().setInputCol(lr.getPredictionCol).setOutputCol("predictionLabel")
// ... the col metadata may only be on the categoryIndexer outputCol though, not the prediction col?
*/

Name: Syntax Error.
Message: 
StackTrace: 

In [16]:
import org.apache.spark.ml.Pipeline

val word_prepare_pipeline = new Pipeline()
    .setStages(Array(
        regexTokenizer,
        stopWordRemover,
        hashingTF,
        
        categoryIndexer
    ))

val word_classify_pipeline = new Pipeline()
    .setStages(Array(
        word_prepare_pipeline,
        lr
    ))

word_classify_pipeline = pipeline_0da008691436


word_prepare_pipeline: org.apache.spark.ml.Pipeline = pipeline_83b38f11669a


pipeline_0da008691436

In [17]:
word_prepare_pipeline

pipeline_83b38f11669a

In [18]:
// Thi is just checking the 'prep' stage of the pipeline works

val newsDataPrepareModel = word_prepare_pipeline.fit(newsTrainDF)
// Note: Still needs fit first (converting it to model) to be able to transform

var newsDataTransformed = newsDataPrepareModel.transform(newsTrainDF)
newsDataTransformed.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|category|               title|         description|                guid|           published|         title_words|       titleFiltered|        hashVectorTF|categoryIndex|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|     AFL|'A baffling decis...|The popular AFL p...|https://www.abc.n...|Wed, 30 Oct 2019 ...|[a, baffling, dec...|[baffling, decisi...|(262144,[119569,1...|          5.0|
|     AFL|'A decision for A...|In his first deta...|https://www.abc.n...|Wed, 24 Jul 2019 ...|[a, decision, for...|[decision, adam, ...|(262144,[4757,191...|          5.0|
|     AFL|'Dangerous and st...|Greater Western S...|https://www.abc.n...|Mon, 09 Sep 2019 ...|[dangerous, and, ...|[dangerous, stupi...|(262

newsDataTransformed = [category: string, title: string ... 7 more fields]


newsDataPrepareModel: org.apache.spark.ml.PipelineModel = pipeline_83b38f11669a


[category: string, title: string ... 7 more fields]

In [19]:
// Lets fit the full model now
val newsDataLRModel = word_classify_pipeline.fit(newsTrainDF)
var newsDataClassified = newsDataLRModel.transform(newsTrainDF)
newsDataClassified.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+
|category|               title|         description|                guid|           published|         title_words|       titleFiltered|        hashVectorTF|categoryIndex|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+
|     AFL|'A baffling decis...|The popular AFL p...|https://www.abc.n...|Wed, 30 Oct 2019 ...|[a, baffling, dec...|[baffling, decisi...|(262144,[119569,1...|          5.0|[-1.4005225499084...|[2.61098719976117...|       5.0|
|     AFL|'A decision for A...|In his first deta...|https://www.abc.n...|Wed, 24 Jul 2019 ...|[a, de

newsDataClassified = [category: string, title: string ... 10 more fields]


newsDataLRModel: org.apache.spark.ml.PipelineModel = pipeline_0da008691436


[category: string, title: string ... 10 more fields]

In [20]:
// Extract the StringIndexerModel labels to work predictions back to string labels
var _stageOne = newsDataLRModel.stages(0).asInstanceOf[org.apache.spark.ml.PipelineModel]
var textIndexLabelerModel = _stageOne.stages(3).asInstanceOf[org.apache.spark.ml.feature.StringIndexerModel]
textIndexLabelerModel.labels

_stageOne: org.apache.spark.ml.PipelineModel = pipeline_83b38f11669a
textIndexLabelerModel: org.apache.spark.ml.feature.StringIndexerModel = strIdx_fadf8e53504a


Array(Rugby Union, World News, Rugby League, Cricket, Soccer Football, AFL, Athletics, Business)

In [21]:
import org.apache.spark.ml.feature.IndexToString

val predictionIdxReverse = new IndexToString().setInputCol("prediction").setOutputCol("predictionLabel")
predictionIdxReverse.setLabels(textIndexLabelerModel.labels)
predictionIdxReverse.transform(newsDataClassified).show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+---------------+
|category|               title|         description|                guid|           published|         title_words|       titleFiltered|        hashVectorTF|categoryIndex|       rawPrediction|         probability|prediction|predictionLabel|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+---------------+
|     AFL|'A baffling decis...|The popular AFL p...|https://www.abc.n...|Wed, 30 Oct 2019 ...|[a, baffling, dec...|[baffling, decisi...|(262144,[119569,1...|          5.0|[-1.4005225499084...|[2.61098719976117...|       5.0|            AFL|
|     AFL|'A decision for A...|In hi

predictionIdxReverse = idxToStr_f195f3e5c1e2


idxToStr_f195f3e5c1e2

In [22]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

//newsDataClassifiedLabelIO = newsDataClassified.select("prediction", "categoryIndex").toDF("predic")

val metrics = new MulticlassClassificationEvaluator().setLabelCol("categoryIndex").setPredictionCol("prediction");
//metrics.setMetric(...)

metrics = mcEval_d145198bee93


mcEval_d145198bee93

In [23]:
s"Training ${metrics.getMetricName}: ${metrics.evaluate(newsDataClassified)}"

Training f1: 0.9800131594163832

In [24]:
// See how well it performs on the dev dataset

var newsDataDevClassified = newsDataLRModel.transform(newsDevDF)

//newsDataDevClassified.show()
predictionIdxReverse.transform(newsDataDevClassified).show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+---------------+
|category|               title|         description|                guid|           published|         title_words|       titleFiltered|        hashVectorTF|categoryIndex|       rawPrediction|         probability|prediction|predictionLabel|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+---------------+
|     AFL|'I just really di...|The prospect of p...|https://www.abc.n...|Tue, 09 Jul 2019 ...|[i, just, really,...|[really, didn, ca...|(262144,[14,34140...|          5.0|[1.07512744835360...|[1.25322584455872...|       3.0|        Cricket|
|     AFL|'I'm so jealous':...|Nat F

newsDataDevClassified = [category: string, title: string ... 10 more fields]


[category: string, title: string ... 10 more fields]

In [25]:
s"Dev ${metrics.getMetricName}: ${metrics.evaluate(newsDataDevClassified)}"

Dev f1: 0.6661677186056167

In [26]:
// use mllib (ie RDD, not ml/DF) for more metrics
import org.apache.spark.mllib.evaluation.MulticlassMetrics

//predictionIdxReverse.transform(newsDataDevClassified) - but will transform after, since it needs doubles
//select should be in form (prediction, label)
val predictionsAndLabels = newsDataDevClassified
    .select("prediction", "categoryIndex")
    .rdd
    .map(row => (row.getDouble(0), row.getDouble(1)) )
;

val metrics = new MulticlassMetrics(predictionsAndLabels)

println(s"Labels: ${textIndexLabelerModel.labels.mkString(", ")}")
val confusionMatrix = metrics.confusionMatrix

Labels: Rugby Union, World News, Rugby League, Cricket, Soccer Football, AFL, Athletics, Business


confusionMatrix = 


predictionsAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[162] at map at <console>:48
metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@158a5cd1
36.0  1.0   1.0   0.0   1.0   1.0   4.0   1.0
1.0   16.0  1.0   1.0   2.0   1.0   3.0   15.0
4.0   0.0   35.0  1.0   1.0   2.0   1.0   0.0
2.0   1.0   0.0   38.0  6.0   1.0   3.0   1.0
3.0   2.0   3.0   2.0   35.0  1.0   2.0   1.0
1.0   0.0   2.0   4.0   1.0   34.0  2.0   1.0
2.0   1.0   0.0   2.0   3.0   1.0   47.0  2.0
3.0   10.0  7.0   6.0   8.0   2.0   1.0   26.0


In [27]:
val newsDevMisClassified = newsDataDevClassified.filter(
    row => row.get(row.fieldIndex("prediction")) != row.get(row.fieldIndex("categoryIndex"))
);

newsDevMisClassified = [category: string, title: string ... 10 more fields]


[category: string, title: string ... 10 more fields]

In [28]:
predictionIdxReverse.transform(newsDevMisClassified)
    .select("category", "title", "description", "predictionLabel", "probability")
    .show(truncate=false)
;

+---------+-------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|category |title                                                                                      |description                                                                                                                                                                                                                                         |predictionLabel|probability                                                                                 

In [29]:
var testing = (spark.createDataFrame(Seq(
    Tuple1("My name is Peter"),
    Tuple1("The temperature has been nice today."),
    Tuple1("Donald Trump is the 2016 US president. 2020 elections are coming up."),
    Tuple1("Scott Morrison, Malcolm Turnbull, and Tony Abbott - the 3 liberal national amigos"),
    Tuple1("To get wickets, you need to pitch the ball up"),
    Tuple1("The wallabies need to score tries to beat the all blacks"),
    Tuple1("Was Johnathon Thurston the greatest league player of all time?"),
    Tuple1("Growth in the retail sector good for the economy")
  )).toDF("text")
);

var testingPrep = testing.withColumn("title", testing.col("text"))


testingPrep = [text: string, title: string]


testing: org.apache.spark.sql.DataFrame = [text: string]


[text: string, title: string]

In [30]:
var testingDataClassified = newsDataLRModel.transform(testingPrep)
//testingDataClassified.show()
predictionIdxReverse.transform(testingDataClassified).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+---------------+
|                text|               title|         title_words|       titleFiltered|        hashVectorTF|       rawPrediction|         probability|prediction|predictionLabel|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+---------------+
|    My name is Peter|    My name is Peter|[my, name, is, pe...|       [name, peter]|(262144,[35119,19...|[0.62289283807962...|[0.00485884335402...|       3.0|        Cricket|
|The temperature h...|The temperature h...|[the, temperature...|[temperature, nic...|(262144,[22346,14...|[-0.3824372491500...|[0.00372563966861...|       1.0|     World News|
|Donald Trump is t...|Donald Trump is t...|[donald, trump, i...|[donald, trump, 2...|(262144,[7612,218...|[-9.0583277573

testingDataClassified = [text: string, title: string ... 6 more fields]


[text: string, title: string ... 6 more fields]

In [31]:
//Optional saving to disk
/*
val model = pipeline.fit(training) //The fit version
pipeline //the unfit version

model.write.overwrite().save("/tmp/spark-logistic-regression-model")
// We can also save the unfit pipeline to disk
pipeline.write.overwrite().save("/tmp/unfit-lr-model")

// And load it back in during production
import org.apache.spark.ml.PipelineModel
val sameModel = PipelineModel.load("/tmp/spark-logistic-regression-model")
*/

Name: Syntax Error.
Message: 
StackTrace: 

In [32]:
//Note they get saved as directories of files
newsDataLRModel.write.overwrite().save("models/news_htf_lr_spark_model")
predictionIdxReverse.write.overwrite().save("models/news_labeler_spark_model")